In [1]:
import sympy as sp
import numpy as np

xc, yc = sp.symbols('x_c y_c')  # x and y values of contact point
xi, eta = sp.symbols(r'xi eta')  # reference coordinate variables
p, n = sp.symbols('p n')  # p is the index of an array and n is the total number of points
xp, yp = sp.Function('x'), sp.Function('y')  # x and y components of a vector
xi_p, eta_p = sp.Function(r'\xi'), sp.Function(r'\eta')  # xi and eta points that make up the boundary of the surface (either 1 or -1 always)

![Figure 1](Figures/Contact%20Point.png)

This demo is provided for constructing the set of non-linear functions to solve for the $\xi$ and $\eta$ reference coordinates of the contact point and construct a Newton-Raphson scheme.

For mapping a reference point $(\xi,\eta)$ to the global/actual position point ($\vec{s}$), we use the following

$$\vec{s}=\sum_{p=0}^{n - 1}\phi_p(\xi,\eta)\vec{s}_p$$

where $\phi_p(\xi,\eta)=\frac{1}{4}(1+\xi_p\xi)(1+\eta_p\eta)$ is the basis/shape function for 2D corresponding to a known reference point $\vec{s}_p$. The position point has components

$$
\vec{s}_p=
\begin{bmatrix}
x_p \\
y_p
\end{bmatrix}
$$

At some contact point $(\xi_c,\eta_c)$, we can set up the following equation below to be analyzed.

In [2]:
phi_p = (sp.Rational(1, 4)*(1 + xi*xi_p(p))*(1 + eta*eta_p(p))).simplify()  # shape function in 2D space
phi_p_func = sp.Function(r'\phi_p')(xi, eta)
s_p = sp.Matrix([xp(p), yp(p)])  # Position vector of surface point
b = sp.Matrix([xc, yc])

eq = sp.Eq(b, sp.Sum(phi_p_func*s_p, (p, 0, n - 1)), evaluate=False)
display(eq)
eq1 = sp.Eq(b, sp.Sum(phi_p*s_p, (p, 0, n - 1)), evaluate=False)
display(eq1)

Eq(Matrix([
[x_c],
[y_c]]), Sum(Matrix([
[\phi_p(xi, eta)*x(p)],
[\phi_p(xi, eta)*y(p)]]), (p, 0, n - 1)))

Eq(Matrix([
[x_c],
[y_c]]), Sum(Matrix([
[(eta*\eta(p) + 1)*(xi*\xi(p) + 1)*x(p)/4],
[(eta*\eta(p) + 1)*(xi*\xi(p) + 1)*y(p)/4]]), (p, 0, n - 1)))

The $x(p)$, $\xi(p)$, and so on should be interpreted as $x_p$, $\xi_p$, and so on. This is how we can use `sympy` to symbolically construct the Newton-Raphson scheme in terms of reference points. For the Newton-Raphson scheme, we have

$$
\begin{bmatrix}
\xi_{i+1} \\
\eta_{i+1} 
\end{bmatrix} = 
\begin{bmatrix}
\xi_i \\
\eta_i
\end{bmatrix} - \bf{J^{-1}}F
$$

In [3]:
# Constructing the vector function F
F = eq1.rhs.doit() - eq1.lhs
F

Matrix([
[-x_c + Sum(eta*xi*\eta(p)*\xi(p)*x(p)/4 + eta*\eta(p)*x(p)/4 + xi*\xi(p)*x(p)/4 + x(p)/4, (p, 0, n - 1))],
[-y_c + Sum(eta*xi*\eta(p)*\xi(p)*y(p)/4 + eta*\eta(p)*y(p)/4 + xi*\xi(p)*y(p)/4 + y(p)/4, (p, 0, n - 1))]])

In [4]:
# Constructing the jacobian J
jac = F.jacobian([xi, eta])
jac

Matrix([
[Sum(eta*\eta(p)*\xi(p)*x(p)/4 + \xi(p)*x(p)/4, (p, 0, n - 1)), Sum(xi*\eta(p)*\xi(p)*x(p)/4 + \eta(p)*x(p)/4, (p, 0, n - 1))],
[Sum(eta*\eta(p)*\xi(p)*y(p)/4 + \xi(p)*y(p)/4, (p, 0, n - 1)), Sum(xi*\eta(p)*\xi(p)*y(p)/4 + \eta(p)*y(p)/4, (p, 0, n - 1))]])

In [5]:
# Construct the inverse jacobian J^{-1}
jac_inv = jac.inv()
jac_inv

Matrix([
[ Sum(xi*\eta(p)*\xi(p)*y(p)/4 + \eta(p)*y(p)/4, (p, 0, n - 1))/(Sum(eta*\eta(p)*\xi(p)*x(p)/4 + \xi(p)*x(p)/4, (p, 0, n - 1))*Sum(xi*\eta(p)*\xi(p)*y(p)/4 + \eta(p)*y(p)/4, (p, 0, n - 1)) - Sum(eta*\eta(p)*\xi(p)*y(p)/4 + \xi(p)*y(p)/4, (p, 0, n - 1))*Sum(xi*\eta(p)*\xi(p)*x(p)/4 + \eta(p)*x(p)/4, (p, 0, n - 1))), -Sum(xi*\eta(p)*\xi(p)*x(p)/4 + \eta(p)*x(p)/4, (p, 0, n - 1))/(Sum(eta*\eta(p)*\xi(p)*x(p)/4 + \xi(p)*x(p)/4, (p, 0, n - 1))*Sum(xi*\eta(p)*\xi(p)*y(p)/4 + \eta(p)*y(p)/4, (p, 0, n - 1)) - Sum(eta*\eta(p)*\xi(p)*y(p)/4 + \xi(p)*y(p)/4, (p, 0, n - 1))*Sum(xi*\eta(p)*\xi(p)*x(p)/4 + \eta(p)*x(p)/4, (p, 0, n - 1)))],
[-Sum(eta*\eta(p)*\xi(p)*y(p)/4 + \xi(p)*y(p)/4, (p, 0, n - 1))/(Sum(eta*\eta(p)*\xi(p)*x(p)/4 + \xi(p)*x(p)/4, (p, 0, n - 1))*Sum(xi*\eta(p)*\xi(p)*y(p)/4 + \eta(p)*y(p)/4, (p, 0, n - 1)) - Sum(eta*\eta(p)*\xi(p)*y(p)/4 + \xi(p)*y(p)/4, (p, 0, n - 1))*Sum(xi*\eta(p)*\xi(p)*x(p)/4 + \eta(p)*x(p)/4, (p, 0, n - 1))),  Sum(eta*\eta(p)*\xi(p)*x(p)/4 + \xi(p)*

In [6]:
# The denominator of each item is messy
# To clean it up, I'm replacing it with a variable "d"
a11 = jac_inv[0, 0]
_, d = sp.fraction(a11)
jac_inv.subs(d, sp.Symbol("d"))

Matrix([
[ Sum(xi*\eta(p)*\xi(p)*y(p)/4 + \eta(p)*y(p)/4, (p, 0, n - 1))/d, -Sum(xi*\eta(p)*\xi(p)*x(p)/4 + \eta(p)*x(p)/4, (p, 0, n - 1))/d],
[-Sum(eta*\eta(p)*\xi(p)*y(p)/4 + \xi(p)*y(p)/4, (p, 0, n - 1))/d,  Sum(eta*\eta(p)*\xi(p)*x(p)/4 + \xi(p)*x(p)/4, (p, 0, n - 1))/d]])

In summary, we have 

$$
F=\left[\begin{matrix}- x_{c} + \sum_{p=0}^{n - 1} \left(\frac{\eta \xi \eta_p \xi_p x_p}{4} + \frac{\eta \eta_p x_p}{4} + \frac{\xi \xi_p x_p}{4} + \frac{x_p}{4}\right)\\- y_{c} + \sum_{p=0}^{n - 1} \left(\frac{\eta \xi \eta_p \xi_p y_p}{4} + \frac{\eta \eta_p y_p}{4} + \frac{\xi \xi_p y_p}{4} + \frac{y_p}{4}\right)\end{matrix}\right]
$$

$$
J=\left[\begin{matrix}\sum_{p=0}^{n - 1} \left(\frac{\eta \eta_p \xi_p x_p}{4} + \frac{\xi_p x_p}{4}\right) & \sum_{p=0}^{n - 1} \left(\frac{\xi \eta_p \xi_p x_p}{4} + \frac{\eta_p x_p}{4}\right)\\\sum_{p=0}^{n - 1} \left(\frac{\eta \eta_p \xi_p y_p}{4} + \frac{\xi_p y_p}{4}\right) & \sum_{p=0}^{n - 1} \left(\frac{\xi \eta_p \xi_p y_p}{4} + \frac{\eta_p y_p}{4}\right)\end{matrix}\right]
$$

$$
J^{-1}=\left[\begin{matrix}\frac{\sum_{p=0}^{n - 1} \left(\frac{\xi \eta_p \xi_p y_p}{4} + \frac{\eta_p y_p}{4}\right)}{d} & - \frac{\sum_{p=0}^{n - 1} \left(\frac{\xi \eta_p \xi_p x_p}{4} + \frac{\eta_p x_p}{4}\right)}{d}\\- \frac{\sum_{p=0}^{n - 1} \left(\frac{\eta \eta_p \xi_p y_p}{4} + \frac{\xi_p y_p}{4}\right)}{d} & \frac{\sum_{p=0}^{n - 1} \left(\frac{\eta \eta_p \xi_p x_p}{4} + \frac{\xi_p x_p}{4}\right)}{d}\end{matrix}\right]
$$

where 
$$
d=\left(\sum_{p=0}^{n - 1} \left(\frac{\eta \eta_p \xi_p x_p}{4} + \frac{\xi_p x_p}{4}\right)\right) \sum_{p=0}^{n - 1} \left(\frac{\xi \eta_p \xi_p y_p}{4} + \frac{\eta_p y_p}{4}\right) - \left(\sum_{p=0}^{n - 1} \left(\frac{\eta \eta_p \xi_p y_p}{4} + \frac{\xi_p y_p}{4}\right)\right) \sum_{p=0}^{n - 1} \left(\frac{\xi \eta_p \xi_p x_p}{4} + \frac{\eta_p x_p}{4}\right)
$$

The code below is the numerical implementation of this scheme.

In [7]:
class Node:
    def __init__(self, pos, ref_pos):
        """
        :param pos: Physical position array 
        :param ref_pos: Reference position array
        """
        self.x, self.y, self.z = pos
        self.xi, self.eta, self.zeta = ref_pos

def get_F(reference_point, physical_point, nodes):
    xi_, eta_ = reference_point
    xc_, yc_ = physical_point
    return np.array([
        0.25*sum([eta_*xi_*p_.eta*p_.xi*p_.x + eta_*p_.eta*p_.x + xi_*p_.xi*p_.x + p_.x for p_ in nodes]) - xc_,
        0.25*sum([eta_*xi_*p_.eta*p_.xi*p_.y + eta_*p_.eta*p_.y + xi_*p_.xi*p_.y + p_.y for p_ in nodes]) - yc_
    ])

def get_jacobian_inverse(reference_point, nodes):
    xi_, eta_ = reference_point
    den = (sum([eta_*p_.eta*p_.xi*p_.x + p_.xi*p_.x for p_ in nodes])/4*sum([xi_*p_.eta*p_.xi*p_.y + p_.eta*p_.y for p_ in nodes])/4 - 
         sum([eta_*p_.eta*p_.xi*p_.y + p_.xi*p_.y for p_ in nodes])/4*sum([xi_*p_.eta*p_.xi*p_.x + p_.eta*p_.x for p_ in nodes])/4)
    return np.array([
        [sum([xi_*p_.eta*p_.xi*p_.y + p_.eta*p_.y for p_ in nodes])/4/den, -sum([xi_*p_.eta*p_.xi*p_.x + p_.eta*p_.x for p_ in nodes])/4/den],
        [-sum([eta_*p_.eta*p_.xi*p_.y + p_.xi*p_.y for p_ in nodes])/4/den, sum([eta_*p_.eta*p_.xi*p_.x + p_.xi*p_.x for p_ in nodes])/4/den]
    ])

def newton_raphson(reference_point, physical_point, nodes, tol=1e-8, max_iter=100):
    xi_, eta_ = reference_point
    for i in range(max_iter):
        F_ = get_F([xi_, eta_], physical_point, nodes)
        jac_inv_ = get_jacobian_inverse([xi_, eta_], nodes)
        xi_, eta_ = np.array([xi_, eta_]) - jac_inv_ @ F_
        if np.linalg.norm(F_) < tol:
            break
    # noinspection PyUnboundLocalVariable
    return np.array([xi_, eta_]), i

Consider a quadrilateral surface bound by the following points:

| Label | $\xi$, $\eta$, $\zeta$ | $x$, $y$, $z$                        |
|-------|------------------------|--------------------------------------|
| 0     | $-1, -1, -1$           | $0.51025339, 0.50683559, 0.99572776$ |
| 1     | $1, -1, -1$            | $1.17943427, 0.69225101, 1.93591633$ |
| 2     | $1, 1, -1$             | $0.99487331, 0.99743665, 2.97094874$ |
| 3     | $-1, 1, -1$            | $0.49444608, 0.99700943, 1.96411315$ |

The contact point is $(0.92088978, 0.74145551, 1.89717136)$. The analysis omits $\zeta$ because we already know that the contact point is on the exterior surface. For this case, $\zeta=-1$. **Note: The implemented procedure needs to use those reference points that are changing.** For example, if contact is on the reference plane $\eta=1$, then the process needs to solve for $\xi$ and $\zeta$.

In [8]:
patch_nodes = [
    Node([0.51025339, 0.50683559, 0.99572776], [-1, -1, -1]),
    Node([1.17943427, 0.69225101, 1.93591633], [1, -1, -1]),
    Node([0.99487331, 0.99743665, 2.97094874], [1, 1, -1]),
    Node([0.49444608, 0.99700943, 1.96411315], [-1, 1, -1])
]

newton_raphson([0.5, -0.5], [0.92088978, 0.74145551], patch_nodes)

(array([ 0.34340497, -0.39835547]), 3)